In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from datetime import datetime
%matplotlib inline

In [22]:
# Loading  data
df = pd.read_csv('UCI_Credit_Card_balanced.csv')

In [23]:
#df = df = df[['X1','X6','X7','X8','X9','X10','X11','X18','X19','X20','X21','X22','X23','Y']] # Correlation
#df = df[['X1','X2','X3','X4','X5','X6','X7','X12','X13','X15','X18','X19','Y']] # Logit SHAP
#df = df[['X1','X2','X3','X4','X6','X7','X8','X12','X18','X19','Y']] # Logit SHAP

In [24]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

In [25]:
scaler = StandardScaler()

In [26]:
scaler.fit(df.drop('Y',axis=1))

StandardScaler()

In [27]:
scaled_features = scaler.fit_transform(df.drop('Y',axis=1))

In [28]:
X = pd.DataFrame(scaled_features,columns=df.columns[:-1]).values
y = df['Y'].values

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [30]:
def create_network(opti = 'sgd'):

    # create model
    model = Sequential()
    
    # hidden layer
    model.add(Dense(X.shape[1], input_dim=X.shape[1], kernel_initializer='normal', activation='relu'))
    #model.add(BatchNormalization())

    # output layer
    model.add(Dense(1, activation='sigmoid'))

    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=opti, metrics=['accuracy'])
    
    # Return compiled network
    return model

In [31]:
# Wrap Keras model so it can be used by scikit-learn
ann = KerasClassifier(build_fn=create_network, verbose=0)

In [32]:
# define search space
param_grid = dict()
param_grid['epochs'] = [50, 75, 100]
param_grid['opti'] = ['sgd', 'adam', 'rmsprop']
param_grid['batch_size'] = [1, 10, 50, 250]

In [33]:
# define search
grid = GridSearchCV(ann, param_grid, scoring='accuracy', n_jobs=-1)

In [19]:
grid_results = grid.fit(X, y, verbose=0)

NameError: name 'X_train' is not defined

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_results.best_score_, grid_results.best_params_))